In [5]:
import os
import numpy as np
import pandas as pd
import h5py
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
import random
from sklearn.metrics import f1_score
from collections import Counter
from tqdm import tqdm
import json

# ==============================
# 설정 파라미터
# ==============================

# HDF5 파일이 저장된 디렉토리
H5_DIR = './data/processed_clips'

# 학습 파라미터
BATCH_SIZE = 512
EPOCHS = 50
HARMONICS_COUNT = 4  # 생성할 고조파의 개수

# GPU 설정 함수
def setup_gpu():
    """
    GPU 설정을 초기화한다.
    """
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
            tf.config.experimental.set_memory_growth(gpus[0], True)
        except RuntimeError as e:
            print(e)

# GPU 설정 실행
setup_gpu()

# ==============================
# 악기 매핑 설정
# ==============================

# MIDI 악기 번호를 인덱스로 매핑
INSTRUMENT_MAPPING = {
    1: 0, 41: 1, 42: 2, 43: 3, 61: 4,
    71: 5, 72: 6, 7: 7, 44: 8, 69: 9, 74: 10
}

# 인덱스를 악기 이름으로 매핑
INSTRUMENT_NAMES = {
    0: "Grand Piano", 1: "Violin", 2: "Viola", 3: "Cello", 4: "Horn",
    5: "Bassoon", 6: "Clarinet", 7: "Harpsichord", 8: "Contrabass",
    9: "Oboe", 10: "Flute"
}

# ==============================
# 유틸리티 함수
# ==============================

def generate_harmonics(note, harmonics=HARMONICS_COUNT):
    """
    MIDI 음높이를 기반으로 고조파 주파수를 생성한다.

    Args:
        note (int): MIDI 음높이 번호.
        harmonics (int): 생성할 고조파의 개수.

    Returns:
        np.ndarray: 고조파 주파수 배열.
    """
    base_frequency = 440 * (2 ** ((note - 69) / 12))  # MIDI 음높이로부터 기본 주파수 계산
    harmonic_frequencies = [base_frequency * (i + 1) for i in range(harmonics)]
    return np.array(harmonic_frequencies, dtype=np.float32)

def load_h5_file(h5_path):
    """
    HDF5 파일에서 데이터를 로드한다.

    Args:
        h5_path (str): HDF5 파일 경로.

    Returns:
        tuple: (clip 데이터, 라벨 배열, 고조파 배열)
    """
    with h5py.File(h5_path, 'r') as f:
        clip = np.array(f['clip'])
        instruments = np.array(f['instrument']).flatten()
        note = np.array(f['note']).flatten()[0]

        # 다중 레이블로 악기별 변환
        labels = np.zeros(len(INSTRUMENT_MAPPING), dtype=np.float32)
        for instr in instruments:
            idx = INSTRUMENT_MAPPING.get(instr, -1)
            if idx != -1:
                labels[idx] = 1

        # 실시간 고조파 생성
        harmonics = generate_harmonics(note, harmonics=HARMONICS_COUNT)

    return clip, labels, harmonics

def calculate_class_weights(h5_files):
    """
    데이터셋 내 각 클래스의 빈도에 따라 클래스 가중치를 계산한다.

    Args:
        h5_files (list): HDF5 파일 경로 리스트.

    Returns:
        dict: 클래스 인덱스와 가중치의 딕셔너리.
    """
    instrument_counts = Counter()
    for file in tqdm(h5_files, desc="클래스 가중치 계산 중"):
        _, labels, _ = load_h5_file(file)
        instrument_indices = np.where(labels == 1)[0]
        instrument_counts.update(instrument_indices)

    total_samples = sum(instrument_counts.values())
    class_weights = {i: total_samples / count for i, count in instrument_counts.items()}
    return class_weights

# ==============================
# 클래스 가중치 저장 및 불러오기 함수
# ==============================

def save_class_weights(class_weights, filepath="class_weights.json"):
    """
    클래스 가중치를 파일로 저장한다.

    Args:
        class_weights (dict): 클래스 인덱스와 가중치의 딕셔너리.
        filepath (str): 저장할 파일 경로.
    """
    class_weights = {int(k): v for k, v in class_weights.items()}
    with open(filepath, 'w') as f:
        json.dump(class_weights, f)
    print(f"클래스 가중치가 '{filepath}' 파일로 저장되었다.")

def load_class_weights(filepath="class_weights.json"):
    """
    파일에서 클래스 가중치를 불러온다.

    Args:
        filepath (str): 불러올 파일 경로.

    Returns:
        dict: 클래스 인덱스와 가중치의 딕셔너리.
    """
    with open(filepath, 'r') as f:
        class_weights = json.load(f)
    print(f"클래스 가중치가 '{filepath}' 파일에서 불러와졌다.")
    return class_weights

# ==============================
# F1 스코어 콜백 클래스
# ==============================

class F1ScoreOnEpochEnd(callbacks.Callback):
    """
    에포크가 끝날 때마다 F1 스코어를 계산하는 콜백 클래스.
    """
    def __init__(self, data_generator):
        super().__init__()
        self.data_generator = data_generator

    def on_epoch_end(self, epoch, logs=None):
        y_true, y_pred = [], []

        for batch in range(len(self.data_generator)):
            [clips, harmonics], labels = self.data_generator[batch]
            predictions = self.model.predict([clips, harmonics], verbose=0)

            y_true.extend(labels)
            y_pred.extend(predictions)

        y_true = np.array(y_true)
        y_pred = np.array(y_pred) >= 0.5

        f1 = f1_score(y_true, y_pred, average='macro')
        print(f"\nEpoch {epoch + 1} - F1 Score: {f1:.4f}")
        logs['f1_score'] = f1

# ==============================
# 데이터 제너레이터 클래스
# ==============================

class H5DataGenerator(tf.keras.utils.Sequence):
    """
    HDF5 파일을 이용한 데이터 제너레이터 클래스.
    """
    def __init__(self, h5_files, batch_size=32, shuffle=True, **kwargs):
        super().__init__(**kwargs)
        self.h5_files = h5_files
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return len(self.h5_files) // self.batch_size

    def __getitem__(self, index):
        batch_files = self.h5_files[index * self.batch_size:(index + 1) * self.batch_size]
        clips, labels, harmonics = [], [], []

        for file in batch_files:
            clip, label, harmonic = load_h5_file(file)
            clips.append(clip)
            labels.append(label)
            harmonics.append(harmonic)

        clips = np.array(clips)
        labels = np.array(labels)
        harmonics = np.array(harmonics)

        return (clips, harmonics), labels

    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.h5_files)

# ==============================
# 학습 및 검증 파일 분리 함수
# ==============================

def split_train_val(h5_dir, val_ratio=0.05):
    """
    HDF5 파일을 학습용과 검증용으로 분리한다.

    Args:
        h5_dir (str): HDF5 파일이 저장된 최상위 디렉토리.
        val_ratio (float): 검증용 데이터의 비율.

    Returns:
        tuple: (학습용 파일 리스트, 검증용 파일 리스트)
    """
    train_files, val_files = [], []

    for root, dirs, files in os.walk(h5_dir):
        h5_files = [os.path.join(root, f) for f in files if f.endswith('.h5')]

        if h5_files:
            val_size = int(len(h5_files) * val_ratio)
            val_files.extend(h5_files[:val_size])
            train_files.extend(h5_files[val_size:])

    print(f"Total training files: {len(train_files)}, Total validation files: {len(val_files)}")
    return train_files, val_files

# ==============================
# 모델 구축 함수
# ==============================

def build_model(input_shape=(256, 46, 1), num_classes=11, harmonics_count=HARMONICS_COUNT):
    """
    멀티 브랜치 CNN 모델을 생성한다.

    Args:
        input_shape (tuple): 스펙트로그램 입력의 형태.
        num_classes (int): 분류할 클래스의 수.
        harmonics_count (int): 고조파의 개수.

    Returns:
        tf.keras.Model: 구축된 Keras 모델.
    """
    # 스펙트로그램 입력 레이어
    spectrogram_input = layers.Input(shape=input_shape, name="spectrogram_input")
    x = spectrogram_input

    # 5번 반복하여 멀티 브랜치 구조 생성
    for i in range(5):
        # 첫 번째 브랜치
        branch1 = layers.Conv2D(64, (11, 1), activation='relu', padding='same')(x)
        branch1 = layers.BatchNormalization()(branch1)
        branch1 = layers.MaxPooling2D((2, 2))(branch1)

        # 두 번째 브랜치
        branch2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        branch2 = layers.BatchNormalization()(branch2)
        branch2 = layers.MaxPooling2D((2, 2))(branch2)

        # 세 번째 브랜치
        branch3 = layers.Conv2D(64, (1, 11), activation='relu', padding='same')(x)
        branch3 = layers.BatchNormalization()(branch3)
        branch3 = layers.MaxPooling2D((2, 2))(branch3)

        # 브랜치 결합
        x = layers.Concatenate()([branch1, branch2, branch3])

    # 스펙트로그램 평탄화
    flatten_spectrogram = layers.Flatten()(x)

    # 고조파 입력 레이어
    harmonics_input = layers.Input(shape=(harmonics_count,), name="harmonics_input")

    # 스펙트로그램과 고조파 결합
    combined = layers.Concatenate()([flatten_spectrogram, harmonics_input])

    # 완전 연결층
    fc1 = layers.Dense(256, activation='relu')(combined)
    fc1 = layers.Dropout(0.5)(fc1)
    fc2 = layers.Dense(128, activation='relu')(fc1)

    # 출력 레이어 (다중 레이블 이진 분류)
    output_layer = layers.Dense(num_classes, activation='sigmoid')(fc2)

    # 모델 생성
    model = models.Model(inputs=[spectrogram_input, harmonics_input], outputs=output_layer)
    return model

# ==============================
# 모델 학습 및 평가 함수
# ==============================

def train_model(h5_dir, batch_size, epochs, class_weights_path="class_weights.json"):
    """
    모델을 학습하고 평가한다.

    Args:
        h5_dir (str): HDF5 파일이 저장된 디렉토리.
        batch_size (int): 배치 크기.
        epochs (int): 학습 에포크 수.
        class_weights_path (str): 클래스 가중치를 저장한 파일 경로.

    Returns:
        tuple: (학습된 모델, 학습 기록)
    """
    train_files, val_files = split_train_val(h5_dir, val_ratio=0.05)
    train_gen = H5DataGenerator(train_files, batch_size=batch_size)
    val_gen = H5DataGenerator(val_files, batch_size=batch_size, shuffle=False)

    print("모델을 구축하는 중...")
    model = build_model(input_shape=(256, 46, 1), num_classes=11, harmonics_count=HARMONICS_COUNT)

    print("모델을 컴파일하는 중...")
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    if os.path.exists(class_weights_path):
        class_weights = load_class_weights(class_weights_path)
    else:
        print("클래스 가중치를 계산하는 중...")
        class_weights = calculate_class_weights(train_files)
        save_class_weights(class_weights, class_weights_path)

    lr_reduce = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1)
    early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)
    f1_on_epoch_end = F1ScoreOnEpochEnd(val_gen)

    class TQDMProgressBar(callbacks.Callback):
        """
        TQDM을 이용하여 학습 진행을 시각화하는 콜백 클래스.
        """
        def on_train_begin(self, logs=None):
            self.epochs = self.params['epochs']
            self.steps = self.params['steps']
            self.tqdm_epoch = tqdm(total=self.epochs, desc="전체 학습 진행", leave=False)

        def on_epoch_begin(self, epoch, logs=None):
            self.tqdm_batch = tqdm(total=self.steps, desc=f"에포크 {epoch+1}/{self.epochs}", leave=False)

        def on_batch_end(self, batch, logs=None):
            self.tqdm_batch.update(1)

        def on_epoch_end(self, epoch, logs=None):
            self.tqdm_batch.close()
            self.tqdm_epoch.update(1)

        def on_train_end(self, logs=None):
            self.tqdm_epoch.close()

    tqdm_callback = TQDMProgressBar()

    print("모델 학습을 시작한다...")
    history = model.fit(train_gen, epochs=epochs,
                        callbacks=[lr_reduce, early_stop, f1_on_epoch_end, tqdm_callback],
                        validation_data=val_gen,
                        class_weight=class_weights,
                        verbose=0)

    return model, history

# ==============================
# 모델 학습 실행
# ==============================

if __name__ == "__main__":
    # 모델 학습
    model, history = train_model(H5_DIR, BATCH_SIZE, EPOCHS)

    # 학습 기록을 DataFrame으로 변환
    history_df = pd.DataFrame(history.history)

    # CSV 파일로 저장
    history_df.to_csv("training_history.csv", index=False)
    print("학습 기록이 'training_history.csv' 파일로 저장되었다.")


Total training files: 693337, Total validation files: 36487
모델을 구축하는 중...
모델을 컴파일하는 중...


클래스 가중치를 계산하는 중...


클래스 가중치 계산 중: 100%|██████████| 693337/693337 [27:43<00:00, 416.90it/s]  


클래스 가중치가 'class_weights.json' 파일로 저장되었다.
모델 학습을 시작한다...


전체 학습 진행:   0%|          | 0/50 [00:00<?, ?it/s]2024-11-10 15:21:55.340431: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


KeyboardInterrupt: 

In [ ]:
# 또는 SavedModel 형식으로 저장
model.save("instrument_classification_model.keras")

In [ ]:
model = load_model('/content/instrument_classification_model.keras')
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ spectrogram_input         │ (None, 256, 46, 1)     │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_15 (Conv2D)        │ (None, 256, 46, 64)    │            768 │ spectrogram_input[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_16 (Conv2D)        │ (None, 256, 46, 64)    │            640 │ spectrogram_input[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_17 (Conv2D)        │ (None, 256, 46, 64)    │            768 │ spectrogram_input[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_15    │ (None, 256, 46, 64)    │            256 │ conv2d_15[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_16    │ (None, 256, 46, 64)    │            256 │ conv2d_16[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_17    │ (None, 256, 46, 64)    │            256 │ conv2d_17[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_15          │ (None, 128, 23, 64)    │              0 │ batch_normalization_1… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_16          │ (None, 128, 23, 64)    │              0 │ batch_normalization_1… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_17          │ (None, 128, 23, 64)    │              0 │ batch_normalization_1… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_6             │ (None, 128, 23, 192)   │              0 │ max_pooling2d_15[0][0… │
│ (Concatenate)             │                        │                │ max_pooling2d_16[0][0… │
│                           │                        │                │ max_pooling2d_17[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_18 (Conv2D)        │ (None, 128, 23, 64)    │        135,232 │ concatenate_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_19 (Conv2D)        │ (None, 128, 23, 64)    │        110,656 │ concatenate_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_20 (Conv2D)        │ (None, 128, 23, 64)    │        135,232 │ concatenate_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_18    │ (None, 128, 23, 64)    │            256 │ conv2d_18[0][0]        │
│ (BatchNormalization) 

 Total params: 5,874,083 (22.41 MB)

 Trainable params: 1,957,387 (7.47 MB)

 Non-trainable params: 1,920 (7.50 KB)

 Optimizer params: 3,914,776 (14.93 MB)

In [19]:
import os
import h5py
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, f1_score, classification_report

# 모델 불러오기
model = load_model('/content/instrument_classification_model.keras')

# 검증 데이터 경로
validation_dir = '/content/processed_clips_test'

# 악기 MIDI 번호에서 인덱스 매핑 정의
instrument_mapping = {
    1: 0, 41: 1, 42: 2, 43: 3, 61: 4,
    71: 5, 72: 6, 7: 7, 44: 8, 69: 9, 74: 10
}
instrument_names = {
    0: "Grand Piano", 1: "Violin", 2: "Viola", 3: "Cello", 4: "Horn",
    5: "Bassoon", 6: "Clarinet", 7: "Harpsichord", 8: "Contrabass",
    9: "Oboe", 10: "Flute"
}

# 고조파 생성 함수 (학습 코드와 동일하게 설정)
H = 4  # 학습 시 사용했던 고조파 개수와 동일하게 설정
def generate_harmonics(note, harmonics=H):
    base_frequency = 440 * (2 ** ((note - 69) / 12))  # MIDI 음높이로부터 기본 주파수 계산
    harmonic_frequencies = [base_frequency * (i + 1) for i in range(harmonics)]
    return np.array(harmonic_frequencies, dtype=np.float32)

# 임계값 설정 (확률이 이 값을 초과하면 악기가 존재한다고 판단)
threshold = 0.5

# 전체 예측 및 실제 레이블 저장 리스트
all_true_labels = []
all_pred_labels = []

# 검증 데이터의 각 클립에 대해 예측 수행
for filename in os.listdir(validation_dir):
    if filename.endswith('.h5'):
        filepath = os.path.join(validation_dir, filename)

        # HDF5 파일 로드
        with h5py.File(filepath, 'r') as hf:
            clip = hf['clip'][:]
            instrument_label = int(hf['instrument'][0])  # 실제 악기 번호

            # 보조 입력 데이터 불러오기 (예: note 정보)
            note = hf['note'][:]

            # 악기 번호가 매핑에 있는지 확인
            if instrument_label in instrument_mapping:
                true_label_vector = np.zeros(len(instrument_mapping))
                true_label_vector[instrument_mapping[instrument_label]] = 1  # 실제 레이블 벡터 생성

                # clip 차원 조정
                clip = clip.reshape(1, 256, 46, 1)

                # 고조파 생성
                harmonics = generate_harmonics(note[0])  # note is (1,), so take note[0]
                harmonics = harmonics.reshape(1, H)  # Reshape to (1, 4) for batch size 1

                # 예측 수행
                prediction = model.predict([clip, harmonics], verbose=0)[0]  # [0] to get first sample

                # 예측 벡터 생성 (임계값 적용)
                pred_label_vector = (prediction >= threshold).astype(int)

                # 전체 레이블 저장
                all_true_labels.append(true_label_vector)
                all_pred_labels.append(pred_label_vector)

# 정확도 및 F1-score 계산
all_true_labels = np.array(all_true_labels)
all_pred_labels = np.array(all_pred_labels)

# 전체 다중 레이블 분류 성능 보고
print("전체 성능:")
print(classification_report(all_true_labels, all_pred_labels, target_names=[instrument_names[i] for i in range(len(instrument_names))]))

# 악기별 성능 분석
for i, instrument_name in instrument_names.items():
    instrument_true = all_true_labels[:, i]
    instrument_pred = all_pred_labels[:, i]

    accuracy = accuracy_score(instrument_true, instrument_pred)
    f1 = f1_score(instrument_true, instrument_pred)

    print(f"{instrument_name} - 정확도: {accuracy * 100:.2f}%, F1-score: {f1:.2f}")


전체 성능:
              precision    recall  f1-score   support

 Grand Piano       1.00      1.00      1.00      1733
      Violin       0.97      0.74      0.84      1364
       Viola       0.43      0.81      0.56       294
       Cello       1.00      0.92      0.95      1255
        Horn       0.64      0.76      0.70       226
     Bassoon       0.88      0.71      0.79       346
    Clarinet       0.95      0.84      0.89       449
 Harpsichord       0.00      0.00      0.00         0
  Contrabass       0.00      0.00      0.00         0
        Oboe       0.00      0.00      0.00         0
       Flute       0.00      0.00      0.00         0

   micro avg       0.90      0.87      0.89      5667
   macro avg       0.53      0.53      0.52      5667
weighted avg       0.94      0.87      0.90      5667
 samples avg       0.87      0.87      0.87      5667

Grand Piano - 정확도: 99.95%, F1-score: 1.00
Violin - 정확도: 93.26%, F1-score: 0.84
Viola - 정확도: 93.38%, F1-score: 0.56
Cello - 정확도

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/me